In [1]:
import pandas as pd

dataset = pd.read_excel('Production_assignment_python.xlsx')
dataset

,period,demand,fixed cost,produuction Cost,storage cost,Capacity
0,period 1,100,2000,40,1,5000
1,period 2,4000,2000,50,1,6000
2,period 3,2500,2000,80,1,4000
3,period 4,4000,2000,60,1,12000
4,period 5,6000,2000,50,1,4000
5,period 6,100,2000,60,1,4000
6,period 7,250,3000,20,1,8000
7,period 8,5000,3000,40,1,4000
8,period 9,6000,3000,90,1,4000
9,period 10,4000,3000,30,1,7000


In [2]:
# generating an index list for the dataframe
indexlist = list(range(1,13))
indexlist

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [3]:
# adding the index list to the dataframe and set is as the index column
dataset['IndexKey'] = indexlist
dataset.set_index('IndexKey', inplace=True)

In [4]:
from pulp import *

# defining a model for Pulp usage
model = LpProblem("model", sense=LpMinimize)

In [5]:
# adding zero to indexlist for inventory variable mapping
index_inventory = [0]+indexlist
print(index_inventory)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [6]:
# define variables
production = LpVariable.dicts('prod', indexlist, lowBound=0, upBound=None, cat='Integer')
inventory = LpVariable.dicts('inventory', index_inventory, lowBound=0, upBound=None, cat="Integer")
binary = LpVariable.dicts('binary', indexlist, lowBound=0, upBound=None, cat='Binary')

inventory[0] = 0

In [7]:
# defining the objective function
model += lpSum(
    [production[i] * dataset.loc[i, 'produuction Cost'] + inventory[i] * dataset.loc[i, 'storage cost'] +
     binary[i] * dataset.loc[i, 'fixed cost'] for i in indexlist])

In [8]:
# defining the constraints
for z in indexlist:
    model += production[z] + inventory[z - 1] - inventory[z] >= dataset.loc[z, 'demand']
    model += production[z] <= binary[z] * dataset.loc[z, 'Capacity']

In [9]:
model.solve()

1

In [10]:
for vv in model.variables():
    print(vv, vv.varValue)

binary_1 1
binary_10 1
binary_11 1
binary_12 0
binary_2 1
binary_3 0
binary_4 1
binary_5 1
binary_6 1
binary_7 1
binary_8 1
binary_9 0
inventory_1 4900
inventory_10 4900
inventory_11 8000
inventory_12 0
inventory_2 6900
inventory_3 4400
inventory_4 2000
inventory_5 0
inventory_6 1150
inventory_7 8900
inventory_8 7900
inventory_9 1900
prod_1 5000
prod_10 7000
prod_11 4000
prod_12 0
prod_2 6000
prod_3 0
prod_4 1600
prod_5 4000
prod_6 1250
prod_7 8000
prod_8 4000
prod_9 0


In [11]:
optimization_data = pd.DataFrame({'demand': dataset['demand'],
    'production': [production[i].varValue for i in production],
    'inventory': [inventory[i].varValue for i in indexlist],
    'opening': [binary[i].varValue for i in binary]
                                  })

In [12]:
optimization_data

,demand,production,inventory,opening
IndexKey,,,,
1,100,5000,4900,1
2,4000,6000,6900,1
3,2500,0,4400,0
4,4000,1600,2000,1
5,6000,4000,0,1
6,100,1250,1150,1
7,250,8000,8900,1
8,5000,4000,7900,1
9,6000,0,1900,0
